## 自動着色

In [1]:
import os
import glob
import math
import random

import numpy as np
import matplotlib.pyplot as plt

import keras.backend as K

from keras.models import Model,Sequential
from keras.optimizers import Adam
from keras.layers import Conv2D, Dense, Input,MaxPooling2D,Lambda,Conv2DTranspose,Activation
from keras.preprocessing.image import load_img, img_to_array,array_to_img,ImageDataGenerator

Using TensorFlow backend.


### データセット読み出し

In [2]:
data_path = 'dataset'
data_lists = glob.glob(os.path.join(data_path, '*jpg'))

### データセットの分割

In [3]:
val_n_sample = math.floor(len(data_lists) * 0.1)
test_n_sample = math.floor(len(data_lists) * 0.1)
train_n_sample = len(data_lists) - val_n_sample - test_n_sample

val_lists = data_lists[:val_n_sample]
test_lists = data_lists[val_n_sample:val_n_sample + test_n_sample]
train_lists = data_lists[val_n_sample + test_n_sample:val_n_sample + test_n_sample + train_n_sample]

### RGBをLABに変換する

In [4]:
import cv2

In [5]:
image_size = 224

def rgb2lab(rgb):
    assert rgb.dtype == 'uint8'
    return cv2.cvtColor(rgb,cv2.COLOR_RGB2Lab)

def lab2rgb(lab):
    assert lab.dtype == 'uint8'
    return cv2.cvtColor(lab,cv2.COLOR_Lab2RGB)

def get__lab_from_data_list(data_list):
    x_lab = []
    for f in data_list:
        rgb = img_to_array(load_img(f,target_size = (image_size,image_size))).astype(np.uint8)
        lab = rgb2lab(rgb)
        x_lab.append(lab)
    return np.stack(x_lab)

### モデルの定義

In [6]:

input = Input(shape=(image_size,image_size,1))

#encoder
e = Conv2D(32,3,strides = 1,padding='same')(input)
e = Activation('relu')(e)
e = Conv2D(64,3,strides = 2,padding='same')(e)
e = Activation('relu')(e)
e = Conv2D(128,3,strides = 2,padding='same')(e)
e = Activation('relu')(e)
e = Conv2D(256,3,strides = 2,padding='same')(e)
e = Activation('relu')(e)

#dncoder

d = Conv2DTranspose(128,3,strides = 2,padding = 'same')(e)
d = Activation('relu')(d)
d = Conv2DTranspose(64,3,strides = 2,padding = 'same')(d)
d = Activation('relu')(d)
d = Conv2DTranspose(32,3,strides = 2,padding = 'same')(d)
d = Activation('relu')(d)
d = Conv2DTranspose(2,1,strides = 1,padding = 'same')(d)
d = Activation('relu')(d)

autoencoder = Model(inputs=input, outputs=d)
autoencoder.compile(optimizer='adam', loss = 'mse')

autoencoder.summary()


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 224, 224, 1)       0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 224, 224, 32)      320       
_________________________________________________________________
activation_1 (Activation)    (None, 224, 224, 32)      0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 112, 112, 64)      18496     
_________________________________________________________________
activation_2 (Activation)    (None, 112, 112, 64)      0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 56, 56, 128)       73856     
_________________________________________________________________
activation_3 (Activation)    (None, 56, 56, 128)       0         
__________

### generator関数の定義

In [7]:
def generator_with_preprocessing(data_list,batch_size,shuffle=False):
    while True:
        if shuffle:
            np.random.shuffle(data_list)
        for i in range(0, len(data_list),batch_size):
            batch_list = data_list[i:i + batch_size]
            batch_lab = get__lab_from_data_list(batch_list)
            batch_l = batch_lab[:,:,:,0:1]
            batch_ab = batch_lab[:,:,:,1:]
            yield(batch_l,batch_ab)

### geneatorの呼び出し

In [8]:
batch_size = 30

train_gen = generator_with_preprocessing(train_lists,batch_size,shuffle=True)
val_gen = generator_with_preprocessing(val_lists,batch_size,shuffle=False)
test_gen = generator_with_preprocessing(test_lists,batch_size,shuffle=False)


train_steps = math.ceil(len(train_lists)/batch_size)
val_steps = math.ceil(len(val_lists)/batch_size)
test_steps = math.ceil(len(test_lists)/batch_size)

### モデルの学習

In [ ]:
epochs = 100

autoencoder.fit_generator(generator=train_gen,steps_per_epoch=train_steps,epochs = epochs,
                          validation_data=val_gen,validation_steps=val_steps)

Epoch 1/100
  1/783 [..............................] - ETA: 4:03:02 - loss: 19055.5391

### モデルの予測

In [10]:
preds = autoencoder.predict_generator(test_gen,steps=test_steps,verbose=0)

x_test = []
y_test = []

for i,(l,ab) in enumerate(generator_with_preprocessing(test_lists,batch_size)):
    x_test.append(l)
    y_test.append(ab)
    if i == (test_steps - 1):
        break
    
x_test = np.vstack(x_test)
y_test = np.vstack(y_test)

SyntaxError: invalid syntax (<ipython-input-10-9689841990b9>, line 3)

### 予測結果をLABからRGBに変換

In [ ]:
test_preds_lab = np.concatenate((x_test,preds),3).astype(np.uint8)

test_preds_lab.shape

test_preds_rgb = []
for i in range(test_preds_lab.shape[0]):
    preds_rgb = lab2rgb(test_preds_lab[i,:,:,:])
    test_preds_rgb.append(preds_rgb)
test_preds_rgb = np.stack(test_preds_rgb)

### 結果表示

In [12]:
from IPython.display import display_png
from PIL import Image,ImageOps

In [ ]:
for i in range(test_preds_rgb.shape[0]):
    gray_image = ImageOps.grayscale(array_to_img(test_preds_rgb[i]))
#     display_png(array_to_img(test_lists[i]))
    display_png(gray_image)
    display_png(array_to_img(test_preds_rgb[i]))
    print('-'*25)
    if i == 5:
        break